In [1]:
from os import environ as env

env['DJANGO_SETTINGS_MODULE'] = 'ov_wag.settings.dev'
env["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
print(env['OV_DB_NAME'])
import django

django.setup()
from wagtail.models import Page

ov = Page.objects.get(id=3)
aapb = Page.objects.get(id=59)

ov


In [2]:
import json

with open('/home/harpo/gbh/aapb/AAPB2/exhibits.json') as f:
    exhibits = json.load(f)
len(exhibits)

26

In [3]:
from cmless.models import Exhibit

exhibits = [Exhibit(**exhibit) for exhibit in exhibits]

In [ ]:
pages = 0
for exhibit in exhibits:
    # exhibit = Exhibit(**exhibit)
    print(exhibit.title, len(exhibit.children))
    pages += len(exhibit.children)
    for n, p in enumerate(exhibit.children):
        print(f'  {n+1}: {p.title} - {p.page}')

pages

TypeError: cmless.models.Exhibit() argument after ** must be a mapping, not Exhibit

In [ ]:
from migrate import (
    parse_cmless_thumbnail,
    parse_featured_markdown,
    download_image,
    markdownify,
)

In [ ]:
from aapb_exhibits.models import AAPBExhibit


def create_exhibit_page(exhibit: Exhibit) -> AAPBExhibit:
    page = AAPBExhibit(
        title=exhibit.title,
        slug=exhibit.slug,
        # authors=exhibit.authors,
        introduction=parse_featured_markdown(exhibit.summary),
        body=[
            ('main', markdownify(exhibit.main) if exhibit.main else None),
            ('extended', markdownify(exhibit.extended) if exhibit.extended else None),
            # ('records', markdownify(exhibit.records) if exhibit.records else None),
        ],
    )
    return page

In [ ]:
for exhibit in exhibits:
    print(f'Creating exhibit: {exhibit.title}')
    page = create_exhibit_page(exhibit)
    aapb.add_child(instance=page)
    page.save_revision().publish()